In [1]:
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [2]:
# import getpass
# import os
# import openai


# import dotenv

# dotenv.load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [3]:
from langchain.globals import set_verbose

set_verbose(True)

In [4]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import dotenv
import os
import uuid
import openai
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
from langchain_community.vectorstores import Qdrant
from langchain.chains import RetrievalQA, ConversationChain
from langchain_community.llms import OpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

dotenv.load_dotenv() # invoke langsmith




True

In [5]:
# app = Flask(__name__)
# app.config['UPLOAD_FOLDER'] = 'uploads/'
# app.config['DATABASE_FOLDER'] = 'database/'
# ALLOWED_EXTENSIONS = {'pdf', 'csv'}

# if not os.path.exists(app.config['UPLOAD_FOLDER']):
#     os.makedirs(app.config['UPLOAD_FOLDER'])

# if not os.path.exists(app.config['DATABASE_FOLDER']):
#     os.makedirs(app.config['DATABASE_FOLDER'])



In [6]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

ALLOWED_EXTENSIONS = {'pdf', 'csv'}

docs = []

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_file(filepath):
    global docs

    _, file_extension = os.path.splitext(filepath)
    file_extension = file_extension.lower()

    print(f"Processing file: {filepath}")
    
    if file_extension == '.pdf':
        loader = PyPDFLoader(filepath)
    elif file_extension == '.csv':
        loader = CSVLoader(filepath, encoding='utf-8')
    else:
        print(f"Unsupported file type: {filepath}")
        return

    docs.extend(loader.load_and_split())

embedding_cache_path = "embedding_cache"
db_collection_names = ["CNS16190_md", "CNS16190-zh_TW_only_provision", "ts_103701v010101p_without_content"]
document_root_path = "docs"
documents = ["CNS16190_correct.pdf", "CNS16190-zh_TW_only_provision.pdf", "ts_103701v010101p_without_content.pdf"]

collection = db_collection_names[0]
if os.path.isdir(os.path.join(embedding_cache_path, "collection", collection)):
    # database already exists, load it
    print("Loading existing database")
    vectorstore = Qdrant.from_existing_collection(
        embedding=OpenAIEmbeddings(),
        path=embedding_cache_path,
        collection_name=collection,
    )
else:
    # database does not exist, create it
    print("Creating new database")
    database_folder = "database"  # 假設你的database文件夾在當前目錄
    for filename in os.listdir(database_folder):
        print(filename) 
        if allowed_file(filename):
            filepath = os.path.join(database_folder, filename)
            process_file(filepath)
            print(f"Processed file: {filename}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    vectorstore = Qdrant.from_documents(
        splits,
        embedding=OpenAIEmbeddings(),
        path=embedding_cache_path,
        collection_name=collection,
    )
retriever = vectorstore.as_retriever()


USER_AGENT environment variable not set, consider setting it to identify your requests.


Loading existing database


In [7]:
vectorstore.similarity_search("""5.4.4.1 Test case 5.4-4-1 (conceptual)
Test purpose
The purpose of this test case is the conceptual assessment of critical security parameters used for integrity and
authenticity checks of software updates and for protection of communication with associated services concerning the
generation mechanisms.
""", k=20)

[Document(metadata={'source': 'database\\ts_103701v010101p_without_content.pdf', 'page': 53, '_id': '9433bb8d096a4b219ce7a0380a21858a', '_collection_name': 'CNS16190_md'}, page_content='The objective of this test group is to assess, firstly, whether the cryptographic methods provide the security guarantees \nthat are necessary for the use case of the communication of critical security parameters and, secondly, whether the cryptographic methods are not known to be vulnerable to a feasible attack. \n5.5.7.1 Test case 5.5-7-1 (conceptual) \nTest purpose \nThe purpose of this test case is the conceptual assessment of the cryptography used for communicating critical security \nparameters via remotely accessible network interfaces. \nTest units \na) For all "Communication Mechanisms", that are remo tely accessible according to  their "Description" in \nIXIT 11-ComMech referenced in any critical secu rity parameter in IXIT 10-SecParam, the TL shall  apply all \ntest units as specified in the 

In [8]:
import pandas as pd


# concat all docs as a string
def format_docs(docs):
    retrieval_docs = "\n\n".join(doc.page_content for doc in docs)
    df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")
    test_data = "\n\n測試資料範例：\n\n"
    for idx in range(3):
        row = df.iloc[idx]
        test_data += f"控制措施{row['provision']}，detail:{row['detail']}，question:{row['Test cenario']}，answer:{row['answer']}\n\n"

    result = retrieval_docs + test_data
    print(result)
    return result

In [9]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4o", request_timeout=60)

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


## Ask llm

In [10]:
df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")

df

,provision,status,support,detail,Test Scenario,answer,reason,result
0,5.1-1,MC,NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN
1,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TL...,NaN,NaN
2,5.2-1,M,NaN,製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資...,漏洞揭露政策明確規定了安全研究人員和其他人報告問題的流程。 此類政策可以根據需要進行更新，以...,通過D-Link 網站（https://support.dlink.com/ReportVu...,NaN,NaN
3,5.3-13,M,NaN,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,NaN,NaN
4,5.3-16,M,NaN,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,NaN,NaN
5,5.4-1,M,NaN,持久存儲中的敏感安全參數應由設備安全的存儲。,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,NaN,NaN
6,5.4-3,M,NaN,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,NaN,NaN
7,5.4-4,M,NaN,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\n管理...,NaN,NaN
8,5.5-1,M,NaN,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN
9,5.5-5,M,NaN,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN


### One row testing

In [11]:
row = df.iloc[7]
query = f"控制措施{row['provision']}，detail:{row['detail']}，provided feature that correspond to provision description:{row['answer']}"
print(
    f"控制措施 {row['provision']}\n\n【detail】\n{row['detail']}\n\n【question】\n{row['Test Scenario']}\n\n【answer】\n{row['answer']}"
)

print(f"\n【Test Scenario】\n{row['Test Scenario']}")



控制措施 5.4-4

【detail】
用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個設備獨有，並且應具有降低對設備類別進行自動攻擊風險的機制。

【question】
為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果使用全球關鍵安全參數，其洩漏可能會導致對其他物聯網設備的大規模攻擊，例如建立殭屍網路。範例：在同一裝置類別的每個裝置上部署不同的對稱金鑰，用於產生和驗證軟體更新的訊息認證碼。範例：設備使用製造商的公鑰來驗證軟體更新。 這不是關鍵的安全參數，並且不需要每個設備都是唯一的。問題：對於設備中用於以下用途的任何關鍵安全參數：- 軟體更新的完整性和真實性檢查；- 保護與設備軟體中相關服務的通訊：1. 這些關鍵安全參數是：A) 每個設備都是唯一的？B) 採用可降低針對各類設備的自動攻擊風險的機制進行生產？

【answer】
D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。
管理和設定通道始終透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。
伺服器證書還可以透過設備的嵌入式證書頒發機構文件進行驗證。

【Test Scenario】
為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果使用全球關鍵安全參數，其洩漏可能會導致對其他物聯網設備的大規模攻擊，例如建立殭屍網路。範例：在同一裝置類別的每個裝置上部署不同的對稱金鑰，用於產生和驗證軟體更新的訊息認證碼。範例：設備使用製造商的公鑰來驗證軟體更新。 這不是關鍵的安全參數，並且不需要每個設備都是唯一的。問題：對於設備中用於以下用途的任何關鍵安全參數：- 軟體更新的完整性和真實性檢查；- 保護與設備軟體中相關服務的通訊：1. 這些關鍵安全參數是：A) 每個設備都是唯一的？B) 採用可降低針對各類設備的自動攻擊風險的機制進行生產？


In [12]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [13]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

few_shot_examples = [
    {
        "question": "根據控制措施5.1-1描述、可對抗控制措施的設備功能與TS 103 701中5.1-1Test Scenario, 生成控制措施5.1-1生成測試場景(Test Scenario)及1~2個問題供測試者檢測可對抗控制措施的設備功能是否有效\
      1.根據CNS16190 控制措施(provision): 5.1-1描述:如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義.\
      2.可對抗控制措施的設備功能(provided feature that correspond to provision description):使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。.",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: 在CNS16190中找出控制措施(Provision)5.1-1 是甚麼?
Intermediate answer: 控制措施(Provision)5.1-1的描述是: 如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義。.
Follow up: 在 TS 103 701中對應於控制措施(provision)的測試場景(test scenario)是甚麼?
Intermediate answer: 5.1.1 Test group 5.1-1 5.1.1.0 Test group objective The test group addresses the provision 5.1-1. This test group addresses all states of the DUT with the exception of factory default. 5.1.1.1 Test case 5.1-1-1 (conceptual) Test purpose The purpose of this test case is the conceptual assessment of the password-based authentication mechanisms. Test units a) The TL shall assess for all password-based user authentication mechanisms in IXIT 1-AuthMech where passwords are not defined by the user according to "Authentication Factor" and used in any state other than the factory default whether the "Password Generation Mechanism" ensures that passwords are unique per device. Assignment of verdict The verdict PASS is assigned if: • each password of a password-based authentication mechanism being used in any state other than the factory default, that is not defined by the user, is unique per device. The verdict FAIL is assigned otherwise. 5.1.1.2 Test case 5.1-1-2 (functional) Test purpose The purpose of this test case is the functional assessment of the password-based authentication mechanisms concerning the completeness of the IXIT documentation a), the passwords defined by the user b) and the generation mechanisms c). Test units a) The TL shall functionally assess whether password-based authentication mechanisms that are not documented in IXIT 1-AuthMech are available via a network interface on the DUT or described in the user manual. EXAMPLE: Network scanning tools allow for discovery of network-based authentication mechanisms. b) For each password-based user authentication mechanism in IXIT 1-AuthMech, the TL shall functionally check whether the user is required to define all passwords that are user-defined according to "Authentication Factor" before being used. c) The TL shall functionally assess whether all passwords of the DUT, that are not defined by the user according to "Authentication Factor" in IXIT 1-AuthMech and used in any state other than the factory default, do not violate the description of the "Password Generation Mechanism". Assignment of verdict The verdict PASS is assigned if: • every discovered password-based authentication mechanism is documented in the IXIT; and • the user is required to define all passwords before being used, that are stated as defined by the user in the IXIT; and • there is no indication that the generation of a not user-defined password of the DUT used in any state other than the factory default differs from the generation mechanism described in the IXIT. The verdict FAIL is assigned otherwise.\
Follow up: 請生成有關 控制措施Provsion: 5.1-1 的測試情境(Test Scenario)。
So the final answer is: 問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置密碼應符合下列其中一項：1. 每個設備都是唯一的。2. 由使用者定義。請詳細說明密碼是如何建立的。\

""",
    },
]
# ,
#     {
#         "question": "根據控制措施5.1-3描述、可對抗控制措施的設備功能與TS 103 701中5.1-3Test Scenario, 生成控制措施5.1-3生成測試場景(Test Scenario)及1~2個問題供測試者檢測可對抗控制措施的設備功能是否有效\
#       1.根據CNS16190 控制措施(provision) 5.1-3描述:用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。.\
#       2.可對抗控制措施的設備功能(provided feature that correspond to provision description):使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。.",
#         "answer": """
# Are follow up questions needed here: Yes.
# Follow up: 在CNS16190中找出控制措施(Provision)5.1-1 是甚麼?
# Intermediate answer: 控制措施(Provision)5.1-1的描述是: 本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TLS v1.2 或更新的連線保護通道，chiper 套件符合 FIPS 14--3雲端管理通訊也透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。 設備與雲端之間透過設備證書進行身份驗證。。.
# Follow up: 在 TS 103 701中對應於控制措施(provision)的測試場景(test scenario)是甚麼?
# Intermediate answer: 5.1.1 Test group 5.1-1 5.1.1.0 Test group objective The test group addresses the provision 5.1-1. This test group addresses all states of the DUT with the exception of factory default. 5.1.1.1 Test case 5.1-1-1 (conceptual) Test purpose The purpose of this test case is the conceptual assessment of the password-based authentication mechanisms. Test units a) The TL shall assess for all password-based user authentication mechanisms in IXIT 1-AuthMech where passwords are not defined by the user according to "Authentication Factor" and used in any state other than the factory default whether the "Password Generation Mechanism" ensures that passwords are unique per device. Assignment of verdict The verdict PASS is assigned if: • each password of a password-based authentication mechanism being used in any state other than the factory default, that is not defined by the user, is unique per device. The verdict FAIL is assigned otherwise. 5.1.1.2 Test case 5.1-1-2 (functional) Test purpose The purpose of this test case is the functional assessment of the password-based authentication mechanisms concerning the completeness of the IXIT documentation a), the passwords defined by the user b) and the generation mechanisms c). Test units a) The TL shall functionally assess whether password-based authentication mechanisms that are not documented in IXIT 1-AuthMech are available via a network interface on the DUT or described in the user manual. EXAMPLE: Network scanning tools allow for discovery of network-based authentication mechanisms. b) For each password-based user authentication mechanism in IXIT 1-AuthMech, the TL shall functionally check whether the user is required to define all passwords that are user-defined according to "Authentication Factor" before being used. c) The TL shall functionally assess whether all passwords of the DUT, that are not defined by the user according to "Authentication Factor" in IXIT 1-AuthMech and used in any state other than the factory default, do not violate the description of the "Password Generation Mechanism". Assignment of verdict The verdict PASS is assigned if: • every discovered password-based authentication mechanism is documented in the IXIT; and • the user is required to define all passwords before being used, that are stated as defined by the user in the IXIT; and • there is no indication that the generation of a not user-defined password of the DUT used in any state other than the factory default differs from the generation mechanism described in the IXIT. The verdict FAIL is assigned otherwise.\
# Follow up: 請生成有關 控制措施Provsion: 5.1-1 的測試情境(Test Scenario)。
# So the final answer is: 問題：5.1.3 Test group 5.1-3 5.1.3.0 Test group objective The test group addresses the provision 5.1-3. According to ETSI TS 103 645 [1]/ETSI EN 303 645 [2] best practice cryptography is defined as cryptography that is suitable for the corresponding use case and has no indication of a feasible attack with current readily available techniques. The objective of this test group is to assess, firstly, whether the cryptographic methods provide the security guarantees that are necessary for the authentication mechanisms and, secondly, whether the cryptographic methods are not known to be vulnerable to a feasible attack.5.1.3.1 Test case 5.1-3-1 (conceptual) Test purpose The purpose of this test case is the conceptual assessment of the cryptography used for the authentication mechanisms concerning the use of best practice cryptography (a-c) and the vulnerability to a feasible attack d). Test units a) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the "Security Guarantees" are appropriate for the use case of user authentication, at least integrity and authenticity are required to be fulfilled. b) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the mechanism according to "Description" is appropriate to achieve the "Security Guarantees". NOTE 1: A holistic approach is required to assess the security of the mechanism. c) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the "Cryptographic Details" are considered as best practice cryptography for the use case of user authentication based on a reference catalogue. If "Cryptographic Details" are not included in a reference catalogue for the corresponding use case (e.g. novel cryptography), the SO shall provide evidences, e.g. a risk analysis, to justify the cryptography is appropriate as best practice for the use case. In such case the TL shall assess whether the evidence is appropriate and reliable for the use case. NOTE 2: A use case based list of examples for best practice cryptography is given in ETSI TR 103 621 [i.7]. Moreover general reference catalogues of best practice cryptography are available, for example:  SOGIS Agreed Cryptographic Mechanisms (https://www.sogis.eu). NOTE 3: A cryptographic algorithm or primitive that is deprecated with regard to its desired security property (e.g. SHA1 for collision resistance) or that relies on a cryptographic parameter (e.g. key-size) that is not appropriate, taking into account the intended lifetime of the DUT and cryptoagility, cannot be considered as best practice cryptography. d) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall assess whether the "Cryptographic Details" are not known to be vulnerable to a feasible attack for the desired security property on the base of the "Security Guarantees" by reference to competent cryptanalytic reports. NOTE 4: Competent cryptanalytic reports are typically published in the scientific literature or, alternatively, are to be provided by the SO. Further, clause D.2 provides information about the expected attack potential for level basic. Assignment of verdict The verdict PASS is assigned if for all user authentication mechanisms: • the security guarantees are appropriate for the use case of user authentication; and • the mechanism is appropriate to achieve the security guarantees with respect to the use case; and • all used cryptographic details are considered as best practice for the use case; and • all used cryptographic details are not known to be vulnerable to a feasible attack for the desired security property. The verdict FAIL is assigned otherwise. 5.1.3.2 Test case 5.1-3-2 (functional) Test purpose The purpose of this test case is the functional assessment of the cryptography used for the authentication mechanisms.Test units a) For each authentication mechanism in IXIT 1-AuthMech used to authenticate users against the DUT, the TL shall functionally assess whether the described "Cryptographic Details" are used by the DUT. EXAMPLE 1: Using a protocol analyser or packet sniffer tool for network-based mechanisms. EXAMPLE 2: If a PKI certificate based authentication is used, sniffing the used certificates and comparing the properties with the described cryptography in the IXIT can be helpful to collect an indication. EXAMPLE 3: If the underlying communication protocol of the authentication mechanism enables different security modes for the communication, trying to downgrade the security mode can be helpful to collect an indication. Assignment of verdict The verdict PASS is assigned if: • there is no indication that any used cryptographic setting differs from its IXIT documentation. The verdict FAIL is assigned otherwise.\
#     範例2生成的測試情境(test scenario):與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通常可以在行動應用程式和裝置之間進行。注意：密碼需要始終受到保護，加密是保護密碼的標準方法。 如果密碼未加密，則它們可以簡單地讀取（即明文）並且很容易洩漏。（範例：傳輸過程中的密碼保護通常可以透過 TLS/SSL 隧道連線或 VPN 進行）。問題：1. 當使用者向裝置進行身份驗證時，身份驗證資料在傳輸過程中如何進行加密保護？\

# """,
#     }

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


# qa_system_prompt = """您是一位了解CNS16190、EN303645和ETSI TS 103 701的有用助手。\
#     使用以下內容來回答最後的問題。如果您不知道答案，只需說您不知道，不要試圖編造答案。
#     當您被要求生成測試情境(test scenario)，請根據以下步驟依序完成任務：
#     1. 根據Context提及的控制措施(provision)和描述(detail)在CNS16190檢索控制措施(provision)內容。\
#     2. 在ETSI TS 103 701中找到對應於控制措施(provision)的測試場景(test scenario)。\
#     3. 根據Context提及的可對抗控制措施的設備功能(provided feature that correspond to provision description)與在ETSI TS 103 701中找到對應於控制措施(provision)的測試場景(test scenario)設計一個Test Scenario，檢測可對抗控制措施的設備功能是否符合控制措施。\
#     4. 並根據生成測試情境(test scenario)撰寫1~3個問題提供測試者檢查(provided feature that correspond to provision description)有效性。\

#     Example:{example}\

    
#     Context: {context}\
#     """
qa_system_prompt = """您是一位了解CNS16190、EN303645和ETSI TS 103 701的有用助手。\
    使用以下內容來回答最後的問題。如果您不知道答案，只需說您不知道，不要試圖編造答案。

    
    Context: {context}\
    """
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

chat_history = []

In [15]:
from langchain_core.messages import HumanMessage



# rag_chain.invoke(f"{query}\n\n針對控制措施、detail和answer從ETSI TS 103 701中對應於控制措施(provision)5.2-1的測試場景(test scenario)")
question = f"{query} 根據控制措施和detail在ETSI TS 103 701中找出最相關Test group 5.4-4，並生成測試流程來供測試者驗證provided feature that correspond to provision description"
# question = f"{query} 1. 說明控制措施(provision)5.2-1:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊。\
#     2. 從ETSI TS 103 701中對應於控制措施(provision)5.2-1的測試場景(test scenario)"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
# ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, "example": few_shot_examples})

chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

# second_question = "What are common ways of doing it?"
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_1["answer"])

根據CNS16190的控制措施5.4-4以及提供的特性描述，我們需要在ETSI TS 103 701 中找到對應的測試組並設計測試流程來驗證所提供的特性。根據控制措施和細節的描述，這涉及用於軟體更新的完整性和真實性檢查，並保護設備軟體中相關服務的通訊的關鍵安全參數應為每個設備獨有，並且具有降低對設備類別進行自動攻擊風險的機制。

### 找到最相關的測試組: ETSI TS 103 701 Test group 5.4.4
根據ETSI TS 103 701的結構，測試組5.4.4應該涵蓋與軟體更新和設備證書相關的測試要求。

### 測試流程:
以下是設計的測試流程，用於驗證D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置，且管理和設定通道始終透過TLS v1.2或更新的連線進行保護，chiper套件符合FIPS 140-3，伺服器證書還可以透過設備的嵌入式證書頒發機構文件進行驗證的特性。

#### 測試步驟:
1. **設備證書唯一性驗證**
    - **步驟1.1:** 檢查每個設備是否配置了唯一的設備證書。
    - **步驟1.2:** 將多個設備連接到管理伺服器，並檢查每個設備提交的證書是否唯一且不同。

2. **TLS 連線保護檢查**
    - **步驟2.1:** 模擬設備與D-Link Cloud的管理和設定通道建立連線。
    - **步驟2.2:** 確認連線使用的TLS版本是否為v1.2或更高版本。
    - **步驟2.3:** 驗證使用的chiper套件是否符合FIPS 140-3標準。

3. **伺服器證書驗證**
    - **步驟3.1:** 在設備端檢查伺服器證書的驗證過程。
    - **步驟3.2:** 驗證伺服器證書是否能夠通過設備嵌入的證書頒發機構文件進行驗證。
    - **步驟3.3:** 確認伺服器證書的有效性和真實性。

4. **設備軟體更新的完整性和真實性檢查**
    - **步驟4.1:** 觸發設備軟體更新過程。
    - **步驟4.2:** 檢查設備是否對更新包進行完整性和真實性檢查。
    - **步驟4.3:** 驗證更新包是否簽名，且設備是否能夠成功驗證其簽名。

5. **自動攻擊風險降低機制**
    - **步驟5.1:*

In [ ]:
second_question = "我現在需要請你幫我生成生成測試場景(Test Scenario)，\
      1.控制措施(provision): 5.2-1, 描述:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資訊：(1-1）收到初始確認;以及 (1-2）狀態更新，直到報告的問題得到解決。.\
      2.可對抗控制措施的設備功能(provided feature that correspond to provision description):通過D-Link 網站（https://support.dlink.com/ReportVulnerabilities.aspx）漏洞通報資訊的聯繫方式向 D-Link 報告安全漏洞，D-Link 會及時分析安全漏洞，如果有必要會向報告者更新安全漏洞的最新狀態。 提供網站通報漏洞郵箱，以接收用戶發現的漏洞，並通過與網站告知使用者漏洞解決進度。."
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history, "example": few_shot_examples})

print(ai_msg_2["answer"])

In [ ]:
# second_question = "我現在需要請你幫我生成生成測試場景(Test Scenario)，\
#       1.條款(provision)為:5.2-1:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資訊：(1-1）收到初始確認;以及 (1-2）狀態更新，直到報告的問題得到解決。.\
#       2.可對抗控制措施的設備功能(provided feature that correspond to provision description):通過D-Link 網站（https://support.dlink.com/ReportVulnerabilities.aspx）漏洞通報資訊的聯繫方式向 D-Link 報告安全漏洞，D-Link 會及時分析安全漏洞，如果有必要會向報告者更新安全漏洞的最新狀態。 提供網站通報漏洞郵箱，以接收用戶發現的漏洞，並通過與網站告知使用者漏洞解決進度。."
# ai_msg_3 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

# print(ai_msg_3["answer"])